# 프로젝트: 없다면 어떻게 될까? (ResNet Ablation Study) 
|평가문항|상세기준|
|:---|:---|
|1. ResNet-34, ResNet-50 모델 구현이 정상적으로 진행되었는가?|블록함수 구현이 제대로 진행되었으며 구현한 모델의 summary가 예상된 형태로 출력되었다.|
|2. 구현한 ResNet 모델을 활용하여 Image Classification 모델 훈련이 가능한가?|tensorflow-datasets에서 제공하는 cats_vs_dogs 데이터셋으로 학습 진행 시 loss가 감소하는 것이 확인되었다.|
|3. Ablation Study 결과가 바른 포맷으로 제출되었는가?|ResNet-34, ResNet-50 각각 plain모델과 residual모델을 동일한 epoch만큼 학습시켰을 때의 validation accuracy 기준으로 Ablation Study 결과표가 작성되었다.|

##  0. Set up

In [4]:
!pip install --upgrade tensorflow_datasets # 4.9.3 최신버전으로 업데이트

     |████████████████████████████████| 5.0 MB 6.6 MB/s            
     |████████████████████████████████| 153 kB 25.9 MB/s            
     |████████████████████████████████| 140 kB 59.5 MB/s            
     |████████████████████████████████| 3.0 MB 43.8 MB/s            
     |████████████████████████████████| 294 kB 61.2 MB/s            
     |████████████████████████████████| 1.0 MB 68.6 MB/s            
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.1
    Uninstalling protobuf-3.19.1:
      Successfully uninstalled protobuf-3.19.1
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.4.0
    Uninstalling tensorflow-datasets-4.4.0:
      Successfully uninstalled tensorflow-datasets-4.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.6.0 requires numpy~=1.19.2, 

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds # tensorflow 데이터셋

print(tf.__version__)
print(np.__version__)
print(tfds.__version__)

2.6.0
1.21.4
4.4.0


In [6]:
# Tensorflow가 활용할 GPU가 장착되어 있는지 확인
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
import urllib3
urllib3.disable_warnings()

#tfds.disable_progress_bar()   # 이 주석을 풀면 데이터셋 다운로드과정의 프로그레스바가 나타나지 않습니다.

(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

DownloadError: Failed to get url https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip. HTTP code: 404.

In [ ]:
# Tensorflow 데이터셋을 로드 후 feature 정보 확인
print(ds_info.features)

In [ ]:
# 데이터의 개수도 확인
print(tf.data.experimental.cardinality(ds_train))
print(tf.data.experimental.cardinality(ds_test))

## 1. Preprocessing

In [ ]:
# 이미지의 표현이 0과 1 사이로 들어오는 정규화 코드
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    image = tf.image.resize(image, (224,224))
    return tf.cast(image, tf.float32) / 255., label

def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [ ]:
num_classes = ds_info.features["label"].num_classes
print(num_classes)

In [ ]:
ds_info.features["label"].names

In [ ]:
# 데이터셋 일부 시각화
fig = tfds.show_examples(ds_train, ds_info)

In [ ]:
# 데이터셋 일부 시각화
fig = tfds.show_examples(ds_test, ds_info)

## 2. Model building

### building block 만들기

In [ ]:
# function for building VGG Block

def build_resnet_block(input_layer,
                    num_cnn=3, 
                    channel=64,
                    block_num=1,
                    is_50=False
                   ):
    # 입력 레이어
    x = input_layer
    
    # conv 레이어
    if is_50 == False: # ResNet34
        # shortcut
        shortcut = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(1,1)
        )(input_layer)
        
        # 디폴트 3개 CNN 레이어
        for cnn_num in range(num_cnn): 
            x = keras.layers.Conv2D(
                    filters=channel,
                    kernel_size=(3,3),
                    padding='same',
                    kernel_initializer='he_normal',
                    name=f'block{block_num}_conv{cnn_num}_1'
                )(x)
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)
            x = keras.layers.Conv2D(
                    filters=channel,
                    kernel_size=(3,3),
                    padding='same',
                    kernel_initializer='he_normal',
                    name=f'block{block_num}_conv{cnn_num}_2'
                )(x)
            x = keras.layers.BatchNormalization()(x)

            # f(x) + x
            x = keras.layers.Add(name=f'block{block_num}_add_{cnn_num}')([shortcut, x])
            x = keras.layers.Activation("relu", name=f'block{block_num}_out_{cnn_num}')(x)
    
    
    else: #ResNet50
        #shortcut
        shortcut = keras.layers.Conv2D(
                filters=channel*4,
                kernel_size=(1,1)
            )(input_layer)
    
        # 디폴트 3개 CNN 레이어
        for cnn_num in range(num_cnn): 

            # 1x1 CNN 레이어
            x = keras.layers.Conv2D(
                        filters=channel,
                        kernel_size=(1,1),
                        kernel_initializer='he_normal',
                        padding='same',
                        name=f'block{block_num}_conv{cnn_num}_1x1'
                    )(x)
            
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)

            # 3x3 CNN 레이어
            x = keras.layers.Conv2D(
                        filters=channel,
                        kernel_size=(3,3),
                        kernel_initializer='he_normal',
                        padding='same',
                        name=f'block{block_num}_conv{cnn_num}_3x3'
                    )(x)
    
            x = keras.layers.BatchNormalization()(x)
            x = keras.layers.Activation('relu')(x)
            
            
            # 1x1 CNN 레이어
            x = keras.layers.Conv2D(
                filters=channel*4,
                kernel_size=(1,1),
                activation='relu',
                kernel_initializer='he_normal',
                padding='same',
                name=f'block{block_num}_conv{cnn_num}_1x1_2'
            )(x)
            
            # 마지막 batch normalization
            x = keras.layers.BatchNormalization()(x)
            
            # f(x) + x
            x = keras.layers.Add(name=f'block{block_num}_add_{cnn_num}')([shortcut, x])
            x = keras.layers.Activation("relu", name=f'block{block_num}_out_{cnn_num}')(x)
    return x

In [ ]:
# block 생성 테스트
resnet_input_layer = keras.layers.Input(shape=(32,32,3))   # 입력 레이어 생성
resnet_block_output = build_resnet_block(resnet_input_layer)    # ResNet 블록 생성

In [ ]:
# 블록 1개짜리 model 생성
model = keras.Model(inputs=resnet_input_layer, outputs=resnet_block_output)
model.summary()

### ResNet 모델 만들기

In [ ]:
# NesNet 모델 자체를 생성하는 함수
def build_resnet(input_shape=(32,32,3),
              num_cnn_list=[3,4,6,3],
              channel_list=[64,128,256,512],
              num_classes=num_classes,
              is_50=False):
    
    assert len(num_cnn_list) == len(channel_list) #모델을 만들기 전에 config list들이 같은 길이인지 확인합니다.
    
    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
#     output = input_layer
    
    # 첫 번째 Conv 레이어
    x = keras.layers.Conv2D(
        filters=64,
        kernel_size=(7,7),
        strides=2
    )(input_layer)
    
    # Max Pooling 레이어
    output = keras.layers.MaxPooling2D(
        pool_size=(3, 3),
        strides=2
    )(x)
    
    # config list들의 길이만큼 반복해서 두 번째 - 다섯 번째 Conv 블록을 생성
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        output = build_resnet_block(
            output,
            num_cnn=num_cnn, 
            channel=channel,
            block_num=i,
            is_50=is_50
        )
    
    output = keras.layers.GlobalAveragePooling2D(name="avg_pool")(output)
    output = keras.layers.Flatten(name='flatten')(output)
    output = keras.layers.Dense(1000, activation='relu', name='fc2')(output)
    output = keras.layers.Dense(1, activation='sigmoid', name='predictions')(output)
    
    
    
    model = keras.Model(
        inputs=input_layer, 
        outputs=output
    )
    return model

In [ ]:
resnet_34 = build_resnet(input_shape=(224, 224, 3), is_50=False)
resnet_34.summary()

In [ ]:
resnet_50 = build_resnet(input_shape=(224, 224, 3), is_50=True)
resnet_50.summary()

In [ ]:
BATCH_SIZE = 32
EPOCH = 10

In [ ]:
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)

In [ ]:
ds_train

## 3. Model fitting

In [ ]:
resnet_34.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_34 = resnet_34.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
resnet_50.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_50 = resnet_50.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

## 4. Results

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history_16.history['loss'], 'r')
plt.plot(history_19.history['loss'], 'b')
plt.title('Model training loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['vgg_16', 'vgg_19'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_16.history['val_accuracy'], 'r')
plt.plot(history_19.history['val_accuracy'], 'b')
plt.title('Model validation accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['vgg_16', 'vgg_19'], loc='upper left')
plt.show()